## Checks NDFD data for NaN values in data
---

In [1]:
import numpy as np
from datetime import datetime, timedelta
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy
from pyproj import Proj

#### Get time rounded down to half hour

In [2]:
def rounded_to_the_last_30_minute():
    now = datetime.now()
    rounded = now - (now - datetime.min) % timedelta(minutes=30)
    return rounded

In [3]:
date = rounded_to_the_last_30_minute()
date

datetime.datetime(2023, 11, 15, 16, 0)

#### Import the data

In [4]:
YYYYMMDD_HHMM = date.strftime('%Y%m%d_%H%M')
YYYYMMDD_HHMM = '20231113_1730'

In [5]:
File = "https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_"+YYYYMMDD_HHMM+".grib2"
File

'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_20231113_1730.grib2'

In [6]:
ds = xr.open_dataset(File)

In [7]:
ds = ds.metpy.parse_cf()
ds = ds.metpy.assign_latitude_longitude(force=False)
ds

<xarray.Dataset>
Dimensions:                                                                   (
                                                                               time: 12,
                                                                               time_bounds_1: 2,
                                                                               time1: 5,
                                                                               time1_bounds_1: 2,
                                                                               time3: 2,
                                                                               ...
                                                                               time4_bounds_1: 2,
                                                                               x: 2145,
                                                                               y: 1377,
                                                                               time2: 43,
                                                                               height_above_ground1: 1,
                                                                               height_above_ground: 1)
Coordinates: (12/13)
    reftime                                                                   datetime64[ns] ...
  * time                                                                      (time) datetime64[ns] ...
  * time1                                                                     (time1) datetime64[ns] ...
  * time3                                                                     (time3) datetime64[ns] ...
  * time4                                                                     (time4) datetime64[ns] ...
  * x                                                                         (x) float32 ...
    ...                                                                        ...
  * time2                                                                     (time2) datetime64[ns] ...
  * height_above_ground1                                                      (height_above_ground1) float32 ...
    metpy_crs                                                                 object ...
  * height_above_ground                                                       (height_above_ground) float32 ...
    latitude                                                                  (y, x) float64 ...
    longitude                                                                 (y, x) float64 ...
Dimensions without coordinates: time_bounds_1, time1_bounds_1, time3_bounds_1,
                                time4_bounds_1
Data variables: (12/20)
    LambertConformal_Projection                                               int32 ...
    time_bounds                                                               (time, time_bounds_1) datetime64[ns] ...
    time1_bounds                                                              (time1, time1_bounds_1) datetime64[ns] ...
    time3_bounds                                                              (time3, time3_bounds_1) datetime64[ns] ...
    time4_bounds                                                              (time4, time4_bounds_1) datetime64[ns] ...
    Dewpoint_temperature_height_above_ground                                  (time2, height_above_ground1, y, x) float32 ...
    ...                                                                        ...
    Total_snowfall_surface_6_Hour_Accumulation                                (time, y, x) float32 ...
    Apparent_Temperature_height_above_ground                                  (time2, height_above_ground1, y, x) float32 ...
    Weather_string_surface                                                    (time2, y, x) float32 ...
    Wind_direction_from_which_blowing_height_above_ground                     (time2, height_above_ground, y, x) float32 ...
    Wind_speed_height_above_ground                                       

In [8]:
x, y = ds.x, ds.y

#### Define function to get gridpoint nearest to a lat and lon

In [9]:
def find_closest(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [10]:
max_temp = ds.Maximum_temperature_height_above_ground_12_Hour_Maximum

#### Define map projection & ETEC gridpoint

In [11]:
proj_data = max_temp.metpy.cartopy_crs
proj_data;

pFull = Proj(proj_data)

In [12]:
siteName = "ETEC"
siteLat, siteLon = (42.68, -73.81) #lat & lon of gridpoint over ETEC
siteX, siteY = pFull(siteLon, siteLat)
siteXidx, siteYidx = find_closest(x, siteX), find_closest(y, siteY)

#### Define max temp Variable

In [13]:
forecastMax = max_temp.isel(x = siteXidx, y = siteYidx).isel()

timeDimMax, vertDimMax = forecastMax.metpy.time.name, forecastMax.metpy.vertical.name

idxTimeMax = slice(None, 2) # First time
idxVertMax = 0 # First (and in this case, only) vertical level

timeDictMax = {timeDimMax: idxTimeMax}
vertDictMax = {vertDimMax: idxVertMax}

forecastMax = forecastMax.isel(vertDictMax).isel(timeDictMax)

#### Define wind speed variable

In [14]:
windSpeed = ds.Wind_speed_height_above_ground
windSpeed = windSpeed.isel(x = siteXidx, y = siteYidx).isel()

timeDimWdsp, vertDimWdsp = windSpeed.metpy.time.name, windSpeed.metpy.vertical.name

idxVertWdsp = 0 # First (and in this case, only) vertical level
vertDictWdsp = {vertDimWdsp: idxVertWdsp}

forecastWindSpeed = windSpeed.isel(vertDictWdsp)

#### Define wind direction variable

In [15]:
windDir = ds.Wind_direction_from_which_blowing_height_above_ground
windDir = windDir.isel(x = siteXidx, y = siteYidx).isel()

timeDimWdr, vertDimWdr = windDir.metpy.time.name, windDir.metpy.vertical.name

idxVertWdr = 0
vertDictWdr = {vertDimWdr: idxVertWdr}

forecastWindDir = windDir.isel(vertDictWdr)

#### Define weather string variable

In [16]:
wx = ds.Weather_string_surface
wx = wx.isel(x = siteXidx, y = siteYidx).isel()

timeDimWx = wx.metpy.time.name

#### Define min temp variable

In [17]:
min_temp = ds.Minimum_temperature_height_above_ground_12_Hour_Minimum

forecastMin = min_temp.isel(x = siteXidx, y = siteYidx).isel()

timeDimMin, vertDimMin = forecastMin.metpy.time.name, forecastMin.metpy.vertical.name

idxTimeMin = slice(None, 2) # First time
idxVertMin = 0 # First (and in this case, only) vertical level

timeDictMin = {timeDimMin: idxTimeMin}
vertDictMin = {vertDimMin: idxVertMin}

forecastMin = forecastMin.isel(vertDictMin).isel(timeDictMin)

#### Define precip probability variable

In [18]:
precip = ds.Total_precipitation_surface_12_Hour_Accumulation_probability_above_0p254

forecastPrecip = precip.isel(x = siteXidx, y = siteYidx).isel()

timeDimPrecip = forecastPrecip.metpy.time.name

idxTimeFull = slice(None, 4) # First 4 times

timeDictPrecip = {timeDimPrecip: idxTimeFull}

forecastPrecip = forecastPrecip.isel(timeDictPrecip)

#### Define cloud cover variable

In [19]:
cloudCover = ds.Total_cloud_cover_surface

forecastCloudCover = cloudCover.isel(x = siteXidx, y = siteYidx).isel()

timeDimCloudCover = forecastCloudCover.metpy.time.name

#### Define Temp variable

In [20]:
temp = ds.Temperature_height_above_ground

forecastTemp = temp.isel(x = siteXidx, y = siteYidx).isel()

timeDimTemp, vertDimTemp = forecastTemp.metpy.time.name, forecastTemp.metpy.vertical.name

idxVertTemp = 0

vertDictTemp = {vertDimTemp: idxVertTemp}

forecastTemp = forecastTemp.isel(vertDictTemp)

#### Define dewpoint variable

In [21]:
dewp = ds.Dewpoint_temperature_height_above_ground

forecastDewp = dewp.isel(x = siteXidx, y = siteYidx).isel()

timeDimDewp, vertDimDewp = forecastDewp.metpy.time.name, forecastDewp.metpy.vertical.name

idxVertDewp = 0

vertDictDewp = {vertDimDewp: idxVertDewp}

forecastDewp = forecastDewp.isel(vertDictDewp)

#### Define relative humidity variable

In [22]:
rh = ds.Relative_humidity_height_above_ground

forecastRH = rh.isel(x = siteXidx, y = siteYidx).isel()

timeDimRH, vertDimRH = forecastRH.metpy.time.name, forecastRH.metpy.vertical.name

idxVertRH = 0

vertDictRH = {vertDimRH: idxVertRH}

forecastRH = forecastRH.isel(vertDictRH)

#### Define time lists at intervals of 1-hr & 12-hr

In [23]:
times1 = forecastCloudCover.metpy.time.values

times12 = forecastPrecip.metpy.time.values

#### Define time lists for max and min temps

In [24]:
timesMax = forecastMax.metpy.time.values

timesMin = forecastMin.metpy.time.values

timesrh = forecastRH.metpy.time

#they get their own time list since they only exist for every other 12-hr period

#### Check cloud cover for NaN

In [25]:
i = 0

cloudNanList=[]

for time_step in times1:

    timeDictCloudCover = {timeDimCloudCover: i}
    forecastCloudCoverNew = forecastCloudCover.isel(timeDictCloudCover)
    
    nanCheck = forecastCloudCoverNew.isnull().values
    
    nanStr = str(nanCheck)
    
    cloudNanList.append(nanStr)
     
    i = i + 1

In [26]:
cloudNanCount = cloudNanList.count('True')

if cloudNanCount > 0:
    cloudNanCheck = True

else:
    cloudNanCheck = False

#### Check wind speed for NaN

In [27]:
i = 0

wdspNanList=[]

for time_step in times1:

    timeDictWdsp = {timeDimWdsp: i}
    wdspNew = forecastWindSpeed.isel(timeDictWdsp)
    
    nanCheck = wdspNew.isnull().values
    
    nanStr = str(nanCheck)
    
    wdspNanList.append(nanStr)
     
    i = i + 1

In [28]:
wdspNanCount = wdspNanList.count('True')

if wdspNanCount > 0:
    wdspNanCheck = True

else:
    wdspNanCheck = False

#### Check wind direction for NaN

In [29]:
i = 0

wdrNanList=[]

for time_step in times1:

    timeDictWdr = {timeDimWdr: i}
    wdrNew = forecastWindDir.isel(timeDictWdr)
    
    nanCheck = wdrNew.isnull().values
    
    nanStr = str(nanCheck)
    
    wdrNanList.append(nanStr)
     
    i = i + 1

In [30]:
wdrNanCount = wdrNanList.count('True')

if wdrNanCount > 0:
    wdrNanCheck = True

else:
    wdrNanCheck = False

#### Check weather string for NaN

In [31]:
i = 0

wxNanList=[]

for time_step in times1:

    timeDictWx = {timeDimWx: i}
    wxNew = wx.isel(timeDictWx)
    
    nanCheck = wxNew.isnull().values
    
    nanStr = str(nanCheck)
    
    wxNanList.append(nanStr)
     
    i = i + 1

In [32]:
wxNanCount = wxNanList.count('True')

if wxNanCount > 0:
    wxNanCheck = True

else:
    wxNanCheck = False

#### Check dewpoint for NaN

In [33]:
i = 0

dewpNanList=[]

for time_step in times1:

    timeDictDewp = {timeDimDewp: i}
    dewpNew = forecastDewp.isel(timeDictDewp)
    
    nanCheck = dewpNew.isnull().values
    
    nanStr = str(nanCheck)
    
    dewpNanList.append(nanStr)
     
    i = i + 1

In [34]:
dewpNanCount = dewpNanList.count('True')

if dewpNanCount > 0:
    dewpNanCheck = True

else:
    dewpNanCheck = False

#### Check temp for NaN

In [35]:
i = 0

tempNanList=[]

for time_step in times1:

    timeDictTemp = {timeDimTemp: i}
    tempNew = forecastTemp.isel(timeDictTemp)
    
    nanCheck = tempNew.isnull().values
    
    nanStr = str(nanCheck)
    
    tempNanList.append(nanStr)
     
    i = i + 1

In [36]:
tempNanCount = tempNanList.count('True')

if tempNanCount > 0:
    tempNanCheck = True

else:
    tempNanCheck = False

#### Check max temp for NaN

In [37]:
i = 0

maxNanList=[]

for time_step in timesMax:

    timeDictMax = {timeDimMax: i}
    maxNew = forecastMax.isel(timeDictMax)
    
    nanCheck = maxNew.isnull().values
    
    nanStr = str(nanCheck)
    
    maxNanList.append(nanStr)
     
    i = i + 1

In [38]:
maxNanCount = maxNanList.count('True')

if maxNanCount > 0:
    maxNanCheck = True

else:
    maxNanCheck = False

#### Check min temp for NaN

In [39]:
i = 0

minNanList=[]

for time_step in timesMin:

    timeDictMin = {timeDimMin: i}
    minNew = forecastMin.isel(timeDictMin)
    
    nanCheck = minNew.isnull().values
    
    nanStr = str(nanCheck)
    
    minNanList.append(nanStr)
     
    i = i + 1

In [40]:
minNanCount = minNanList.count('True')

if minNanCount > 0:
    minNanCheck = True

else:
    minNanCheck = False

#### Check RH for NaN

In [41]:
i = 0

rhNanList=[]

for time_step in timesrh:

    timeDictRH = {timeDimRH: i}
    rhNew = forecastRH.isel(timeDictRH)
    
    nanCheck = rhNew.isnull().values
    
    nanStr = str(nanCheck)
    
    rhNanList.append(nanStr)
     
    i = i + 1

In [42]:
rhNanCount = rhNanList.count('True')

if rhNanCount > 0:
    rhNanCheck = True

else:
    rhNanCheck = False

#### Display whether there is a NaN value or Not for each variable

In [43]:
cloudNanCheck

True

In [44]:
wdspNanCheck

True

In [45]:
wdrNanCheck

True

In [46]:
wxNanCheck

True

In [47]:
dewpNanCheck

False

In [48]:
tempNanCheck

False

In [49]:
maxNanCheck

False

In [50]:
minNanCheck

False

In [51]:
rhNanCheck

False